In [ ]:
# Code to reproduce figure 2c in Forlin, Mebrahtu et al

In [1]:
import warnings
warnings.filterwarnings("ignore")
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc

from itertools import combinations
from statsmodels.stats.multitest import multipletests
from scipy.stats import ranksums


In [3]:
data_path = #insert data path here
adata = sc.read(data_path)
sc.pp.normalize_total(adata, target_sum = 1e3)


In [4]:
df_cytos = pd.read_csv('./data/cytokine_genelist.csv', index_col = [0])
hallmark_ifna = ast.literal_eval(df_cytos.loc[7, 'genes'])
hallmark_ifnb = ast.literal_eval(df_cytos.loc[8, 'genes'])



In [6]:
hallmark_list = [hallmark_ifna, hallmark_ifnb]

for i, hallmark in enumerate(['hallmark_ifna', 'hallmark_ifnb']):
    genes = hallmark_list[i]
    df_bootstrapped = pd.DataFrame(columns = ['mean_value', 'CellType', 'treatment'])
    for st in list(set(adata.obs.stim_treat_time)):
        temp = adata[adata.obs.stim_treat_time == st]
        for ct in ['B', 'CD4T', 'CD8T', 'Monocyte', 'DC', 'Eosinophil', 'NK', 'Neutrophil', 'pDC', 'Plasmablast']:
            temp2 = temp[temp.obs.CellType == ct]
            temp2 = temp2[:, temp2.var.index.isin(genes)].to_df()
            
            summ = np.sum(temp2, axis = 1)
            df_ctrl = adata[(adata.obs.stim_treat_time == 'IFNb_Ctrl_2h')&(adata.obs.CellType == ct), adata.var.index.isin(genes)].to_df()
            ctrl_summ = np.sum(df_ctrl, axis = 1)
            
            for i in range(100):
                mean_value = np.mean(summ.sample(len(summ), replace = True))
                df_bootstrapped.loc[st + '_' + ct + '_' + str(i)] = [mean_value, ct, st]
    df_bootstrapped = df_bootstrapped.reset_index().drop('index', axis = 1)
    df_bootstrapped['hallmark'] = hallmark
    #df_bootstrapped.to_csv(f'./data/Figure2_{hallmark}_toR.csv', sep = ',')
            

In [7]:
df_bootstrapped

,mean_value,CellType,treatment,hallmark
0,8.473472,B,IFNb_MOM1000_2h,hallmark_ifnb
1,8.000530,B,IFNb_MOM1000_2h,hallmark_ifnb
2,7.831872,B,IFNb_MOM1000_2h,hallmark_ifnb
3,8.207777,B,IFNb_MOM1000_2h,hallmark_ifnb
4,7.996759,B,IFNb_MOM1000_2h,hallmark_ifnb
...,...,...,...,...
6995,9.403514,Plasmablast,IFNb_ABR1000_2h,hallmark_ifnb
6996,9.310530,Plasmablast,IFNb_ABR1000_2h,hallmark_ifnb
6997,8.384607,Plasmablast,IFNb_ABR1000_2h,hallmark_ifnb
6998,10.273859,Plasmablast,IFNb_ABR1000_2h,hallmark_ifnb


In [18]:
# Define hallmark gene sets and names
hallmark_list = [hallmark_ifna, hallmark_ifnb]
hallmark_names = ['hallmark_ifna', 'hallmark_ifnb']

# Store results
all_vs_all_results = []

# Loop over hallmark sets
for hallmark_genes, hallmark_name in zip(hallmark_list, hallmark_names):
    
    for ct in ['B', 'CD4T', 'CD8T', 'Monocyte', 'DC', 'Eosinophil', 'NK', 'Neutrophil', 'pDC', 'Plasmablast']:
        
        # Filter to conditions where the cell type exists
        conditions = set(
            adata.obs[adata.obs.CellType == ct]['stim_treat_time']
        )
        
        # Compare all pairs of conditions
        for cond1, cond2 in combinations(conditions, 2):
            # Get data for cond1
            df1 = adata[(adata.obs.stim_treat_time == cond1) & 
                        (adata.obs.CellType == ct), 
                        adata.var.index.isin(hallmark_genes)].to_df()
            df1_summed = df1.sum(axis=1)

            # Get data for cond2
            df2 = adata[(adata.obs.stim_treat_time == cond2) & 
                        (adata.obs.CellType == ct), 
                        adata.var.index.isin(hallmark_genes)].to_df()
            df2_summed = df2.sum(axis=1)

            # Skip if empty
            if df1_summed.empty or df2_summed.empty:
                continue

            # Perform Wilcoxon rank-sum test
            stat, pval = ranksums(df1_summed, df2_summed, alternative = 'two-sided')

            # Store result
            all_vs_all_results.append({
                'hallmark': hallmark_name,
                'CellType': ct,
                'Condition1': cond1,
                'Condition2': cond2,
                'Statistic': stat,
                'p-value': pval,
                'mean1': df1_summed.mean(),
                'mean2': df2_summed.mean()
            })

# Convert to DataFrame
all_vs_all_df = pd.DataFrame(all_vs_all_results)

all_vs_all_df['p_adj'] = multipletests(all_vs_all_df['p-value'], method='bonferroni')[1]


In [19]:
all_vs_all_df.to_csv('./extended_data/Figure2_extended_data.csv')